# Multilingual E-commerce Content Classifier
This notebook implements a binary classifier to detect e-commerce vs non-e-commerce content in multiple languages.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import pandas as pd
import numpy as np

In [ ]:
# Load pre-trained multilingual model
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=2,
    ignore_mismatched_sizes=True  # Required when changing number of labels
)

In [ ]:
data = {
    'text': [
        # ---------------------------
        # E-commerce (Label = 1) - 500 Examples
        # ---------------------------
        # English E-commerce
        'best deals on laptops online',
        'how to checkout my shopping cart',
        'discount codes for online shopping',
        'product reviews for smartphones',
        'where to buy headphones cheap',
        'add to cart button not working',
        'compare prices for washing machines',
        'best e-commerce sites for clothes',
        'how to apply promo code at checkout',
        'free shipping offers today',
        'where can I sell my old phone online',
        'best budget smartphones 2024',
        'how to track my online order',
        'return policy for Amazon orders',
        'why is my payment failing on Daraz',
        'best time to buy electronics online',
        'how to contact Flipkart customer care',
        'cash on delivery options in Bangladesh',
        'best online grocery stores in Dhaka',
        'how to cancel an order on AjkerDeal',
        'where to buy authentic cosmetics online',
        'best online bookstore in Bangladesh',
        'how to check product authenticity online',
        'cheapest place to buy mobile accessories',
        'how to get refund for online purchase',

        # Bangla E-commerce (বাংলা ই-কমার্স উদাহরণ)
        'অনলাইনে ল্যাপটপ কিনতে চাই',
        'শপিং কার্টে কীভাবে পেমেন্ট করব?',
        'ডিসকাউন্ট কোড কোথায় পাবো?',
        'স্মার্টফোনের রিভিউ পড়তে চাই',
        'সস্তায় হেডফোন কোথায় পাব?',
        'কার্টে যোগ করুন বাটন কাজ করছে না',
        'ওয়াশিং মেশিনের দাম কোথায় কম?',
        'অনলাইনে জামা কিনতে ভালো সাইট',
        'প্রোমো কোড কিভাবে ব্যবহার করব?',
        'আজকে ফ্রি শিপিং অফার আছে?',
        'আমার পুরোনো ফোন অনলাইনে বিক্রি করব কোথায়?',
        '২০২৪ সালের সস্তার স্মার্টফোন কোনটা?',
        'আমার অর্ডার ট্র্যাক কিভাবে করব?',
        'Amazon থেকে অর্ডার রিটার্ন নীতিমালা কী?',
        'Daraz-এ পেমেন্ট কেন হচ্ছে?',
        'ইলেকট্রনিক্স কিনতে সময় কোনটা?',
        'Flipkart-এ কাস্টমার কেয়ার?',
        'বাংলাদেশে ক্যাশ অন ডেলিভারি অপশন আছে?',
        'ঢাকায় ভালো অনলাইন কোনটা?',
        'AjkerDeal-এ অর্ডার বাতিল করব?',
        'অনলাইনে আসল প্রসাধনী কোথায় পাব?',
        'বাংলাদেশের অনলাইন বইয়ের দোকান?',
        'অনলাইনে পণ্যের যাচাই করব?',
        'মোবাইল এক্সেসরিজ সবচেয়ে সস্তায় কোথায়?',
        'অনলাইন কেনার টাকা ফেরত পাব?',

        # ---------------------------
        # Non-E-commerce (Label = 0) - 500 Examples
        # ---------------------------
        # English Non-E-commerce
        'what is the capital of France?',
        'how to make chocolate cake at home',
        'latest news about climate change',
        'Python programming tutorials for beginners',
        'weather forecast for tomorrow in Dhaka',
        'history of ancient Egyptian civilization',
        'how to improve your communication skills',
        'best universities for computer science',
        'how to treat a common cold at home',
        'latest discoveries in space exploration',
        'how to grow tomatoes in your backyard',
        'what causes earthquakes and how to prepare',
        'how to learn Bangla language quickly',
        'tips for traveling to Cox\'s Bazar',
        'how to write a professional CV',
        'best exercises for weight loss at home',
        'how to start meditation for beginners',
        'what are the symptoms of dengue fever',
        'how to repair a bicycle puncture',
        'best books to read for self-improvement',
        'how to reduce stress and anxiety naturally',
        'what is artificial intelligence and its uses',
        'how to take care of indoor plants',
        'best tourist places in Bangladesh',
        'how to make a simple website using HTML',

        # Bangla Non-E-commerce (বাংলা সাধারণ উদাহরণ)
        'ফ্রান্সের রাজধানী কোথায়?',
        'ঘরে চকলেট কেক বানানোর পদ্ধতি',
        'জলবায়ু পরিবর্তন সম্পর্কে সর্বশেষ খবর',
        'পাইথন প্রোগ্রামিং শেখার টিউটোরিয়াল',
        'আগামীকাল ঢাকার আবহাওয়া কেমন হবে?',
        'প্রাচীন মিশরীয় সভ্যতার ইতিহাস',
        'কিভাবে আপনার যোগাযোগ দক্ষতা উন্নত করবেন',
        'কম্পিউটার সায়েন্সের জন্য ',
        'ঘরেই সর্দি-কাশির চিকিৎসা করব',
        'মহাকাশ গবেষণায় সাম্প্রতিক আবিষ্কার',
        'বাড়ির আঙিনায় টমেটো চাষ ',
        'ভূমিকম্পের কারণ ও প্রস্তুতি কিভাবে নেব',
        'দ্রুত বাংলা ভাষা শেখার উপায়',
        'কক্সবাজার ভ্রমণের টিপস',
        'একটি পেশাদার CV লিখনের নিয়ম',
        'ওজন কমানোর জন্য ভালো ব্যায়াম',
        'meditation শুরু করবেন কিভাবে ?',
        'ডেঙ্গু জ্বরের লক্ষণ কী কী?',
        'সাইকেলের পাংচার মেরাম',
        'আত্মউন্নয়নের জন্য বই',
        'stress এবং anxiety কমানোর প্রাকৃতিক উপায়',
        'কৃত্রিম বুদ্ধিমত্তা কী এবং এর ব্যবহার',
        'ঘরের গাছের যত্ন কিভাবে নেবেন',
        'বাংলাদেশের সেরা পর্যটন স্থান',
        'HTML ব্যবহার করে ওয়েবসাইট বানানোর পদ্ধতি',
    ],
    'label': [1]*50 + [0]*50  # 500 e-commerce (1), 500 non-e-commerce (0)
}

In [ ]:
def prepare_dataset(data):
    dataset = Dataset.from_dict(data)
    tokenized_dataset = dataset.map(
        lambda x: tokenizer(x['text'], padding='max_length', truncation=True, max_length=128),
        batched=True
    )
    return tokenized_dataset

In [ ]:
# Prepare and split dataset
dataset = prepare_dataset(data)
train_dataset = dataset.shuffle().select(range(len(dataset) - 2))  # Leave 2 out for eval
eval_dataset = dataset.select(range(len(dataset) - 2, len(dataset)))

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# Training arguments optimized for e-commerce classification
training_args = TrainingArguments(
    output_dir='./ecommerce_model',
    num_train_epochs=5,
    per_device_train_batch_size=4,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch"
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Train the model
trainer.train()

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/125 [00:00<?, ?it/s]

{'loss': 0.6536, 'grad_norm': 4.787451267242432, 'learning_rate': 5e-06, 'epoch': 0.4}
{'loss': 0.6072, 'grad_norm': 3.3685834407806396, 'learning_rate': 1e-05, 'epoch': 0.8}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.6053858995437622, 'eval_runtime': 0.2372, 'eval_samples_per_second': 8.431, 'eval_steps_per_second': 4.216, 'epoch': 1.0}
{'loss': 0.5287, 'grad_norm': 6.964407920837402, 'learning_rate': 1.5e-05, 'epoch': 1.2}
{'loss': 0.3846, 'grad_norm': 29.001192092895508, 'learning_rate': 2e-05, 'epoch': 1.6}
{'loss': 0.1922, 'grad_norm': 95.23049926757812, 'learning_rate': 2.5e-05, 'epoch': 2.0}


...

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.0006136203301139176, 'eval_runtime': 0.2809, 'eval_samples_per_second': 7.121, 'eval_steps_per_second': 3.56, 'epoch': 4.0}
{'loss': 0.0007, 'grad_norm': 0.014048384502530098, 'learning_rate': 3e-05, 'epoch': 4.4}
{'loss': 0.0006, 'grad_norm': 0.013844628818333149, 'learning_rate': 1e-05, 'epoch': 4.8}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.00046611885773018, 'eval_runtime': 0.3035, 'eval_samples_per_second': 6.591, 'eval_steps_per_second': 3.295, 'epoch': 5.0}
{'train_runtime': 298.8529, 'train_samples_per_second': 1.64, 'train_steps_per_second': 0.418, 'train_loss': 0.19425887713581325, 'epoch': 5.0}


TrainOutput(global_step=125, training_loss=0.19425887713581325, metrics={'train_runtime': 298.8529, 'train_samples_per_second': 1.64, 'train_steps_per_second': 0.418, 'total_flos': 32231104281600.0, 'train_loss': 0.19425887713581325, 'epoch': 5.0})

In [ ]:
def classify_ecommerce(text):
    # Prepare the text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    
    # Get prediction
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.softmax(outputs.logits, dim=-1)
        ecommerce_score = predictions[0][1].item()
        non_ecommerce_score = predictions[0][0].item()
        predicted_class = torch.argmax(predictions, dim=-1).item()
    
    return {
        "text": text,
        "classification": "E-commerce" if predicted_class == 1 else "Non-e-commerce",
        "ecommerce_probability": f"{ecommerce_score:.2%}",
        "confidence": f"{max(ecommerce_score, non_ecommerce_score):.2%}"
    }

In [ ]:
# Save the model
model.save_pretrained('./ecommerce_model')
tokenizer.save_pretrained('./ecommerce_model')

('./ecommerce_model\\tokenizer_config.json',
 './ecommerce_model\\special_tokens_map.json',
 './ecommerce_model\\vocab.txt',
 './ecommerce_model\\added_tokens.json',
 './ecommerce_model\\tokenizer.json')

## Test the E-commerce Classifier

In [ ]:
test_cases = [
    # English (50) ========================================
    # E-Commerce (25)
    "Best wireless headphones under $50",
    "Where to buy organic tea online?",
    "iPhone 15 Pro Max deals with contract",
    "Discount codes for Adidas running shoes",
    "Top gaming laptops under $1000",
    "Cheapest flights from NYC to London",
    "Best online jewelry stores for engagement rings",
    "How to return a defective product on Amazon?",
    "Best Android phones under $300",
    "Sell old PlayStation 5 for cash",
    "Top-rated vacuum cleaners 2024",
    "Same-day grocery delivery services",
    "Best mirrorless camera for photography",
    "Price comparison for Samsung Galaxy S24",
    "Black Friday deals on home appliances",
    "Where to buy luxury watches online?",
    "How to check Flipkart order status?",
    "Affordable ergonomic office chairs",
    "Best books for learning Python programming",
    "Authentic designer handbags online stores",
    "Last-minute hotel deals in Miami",
    "Buy refurbished MacBooks with warranty",
    "Best sites to sell handmade crafts",
    "Where to buy vegan protein powder?",
    "Cheap car rental discounts this weekend",

    # Non-E-Commerce (25)
    # General/Neutral (15)
    "How to bake a chocolate cake?",
    "Latest news on climate change",
    "Python vs. JavaScript for beginners",
    "Symptoms of vitamin D deficiency",
    "History of the Roman Empire",
    "How to improve public speaking skills?",
    "Best exercises for weight loss",
    "Today's weather in Tokyo",
    "How to write a resume for a job?",
    "Difference between AI and machine learning",
    "Why do cats purr?",
    "Funny memes about Monday mornings",
    "How to tie a tie step by step?",
    "Best movies to watch on Netflix",
    "Signs your partner is cheating",

    # Offensive/Harmful (5)
    "How to hack someone's Facebook account?",
    "Where to buy illegal drugs online?",
    "Make a bomb with household items",
    "Hate speech against religious minorities",
    "How to scam people for money?",

    # Random/Other (5)
    "How to get rid of cockroaches?",
    "Most expensive paintings in the world",
    "How to apologize after a fight?",
    "Why is the sky blue?",
    "Tips for traveling solo in Europe",

    # Bengali (50) ========================================
    # E-Commerce (25)
    "অনলাইনে সেরা হেডফোন কিনতে চাই",
    "জৈব চা কোথায় কিনবেন অনলাইনে?",
    "আইফোন ১৫ প্রো ম্যাক্স ডিসকাউন্টে",
    "নাইকি জুতার ডিসকাউন্ট কোড",
    "১০০০ ডলারের মধ্যে সেরা গেমিং ল্যাপটপ",
    "ঢাকা থেকে লন্ডনের সস্তা ফ্লাইট",
    "অনলাইনে বিশ্বস্ত জুয়েলারি দোকান",
    "আমাজনে অর্ডার ক্যানসেল করার নিয়ম",
    "৩০০ ডলারের মধ্যে সেরা অ্যান্ড্রয়েড ফোন",
    "পুরানো প্লেস্টেশন ৫ বিক্রি করার সাইট",
    "২০২৪ সালের সেরা ভ্যাকুয়াম ক্লিনার",
    "ঘরে বসে মুদিখানা ডেলিভারি সার্ভিস",
    "ফটোগ্রাফির জন্য সেরা মিররলেস ক্যামেরা",
    "স্যামসাং গ্যালাক্সি এস২৪ এর দাম তুলনা",
    "ব্ল্যাক ফ্রাইডেতে এসি ও ফ্রিজের অফার",
    "অনলাইনে আসল রোলেক্স ঘড়ি কোথায় পাবো?",
    "ফ্লিপকার্ট অর্ডার ট্র্যাক করার উপায়",
    "সস্তায় আরগোনমিক অফিস চেয়ার",
    "পাইথন শেখার সেরা বই",
    "অনলাইনে অথেন্টিক গুচি ব্যাগ কিনবেন কোথায়?",
    "ডিসেম্বরে দার্জিলিং হোটেল ডিসকাউন্ট",
    "ওয়ারেন্টি সহ রিফার্বিশড ম্যাকবুক",
    "হস্তশিল্প বিক্রির সেরা ওয়েবসাইট",
    "ভেগান প্রোটিন পাউডার কিনতে চাই",
    "সপ্তাহান্তে গাড়ি ভাড়ার ছাড়",

    # Non-E-Commerce (25)
    # General/Neutral (15)
    "চকোলেট কেক বানানোর সহজ উপায়",
    "জলবায়ু পরিবর্তনের সর্বশেষ খবর",
    "প্রোগ্রামিং শিখতে পাইথন নাকি জাভাস্ক্রিপ্ট?",
    "ভিটামিন ডি ঘাটতির লক্ষণ কী?",
    "রোমান সাম্রাজ্যের ইতিহাস",
    "পাবলিক স্পিকিং কিভাবে উন্নত করবেন?",
    "ওজন কমানোর সেরা ব্যায়াম",
    "আজকে টোকিওর আবহাওয়া কেমন?",
    "চাকরির জন্য সিভি怎么写?",
    "এআই এবং মেশিন লার্নিংয়ের পার্থক্য",
    "বিড়াল কেন গড়গড় করে?",
    "সোমবার নিয়ে মজার মিমস",
    "টাই বাঁধার সহজ পদ্ধতি",
    "নেটফ্লিক্সে দেখার মত সিনেমা",
    "প্রেমিক/প্রেমিকা কি প্রতারণা করছে?",

    # Offensive/Harmful (5)
    "কারো ফেসবুক অ্যাকাউন্ট হ্যাক করার উপায়",
    "অনলাইনে অবৈধ মাদক কোথায় কিনতে পাওয়া যায়?",
    "ঘরোয়া জিনিস দিয়ে বোমা বানানোর পদ্ধতি",
    "ধর্মীয় সংখ্যালঘুদের বিরুদ্ধে ঘৃণামূলক বক্তব্য",
    "মানুষকে প্রতারণা করে টাকা আনার উপায়",

    # Random/Other (5)
    "তেলাপোকা দূর করার উপায়",
    "বিশ্বের সবচেয়ে দামি পেন্টিং",
    "ঝগড়ার পরে ক্ষমা চাওয়ার নিয়ম",
    "আকাশ নীল কেন হয়?",
    "ইউরোপে একা ভ্রমণের টিপস"
]

for test in test_cases:
    result = classify_ecommerce(test)
    print(f"\nText: {result['text']}")
    print(f"Classification: {result['classification']}")
    print(f"E-commerce Probability: {result['ecommerce_probability']}")
    print(f"Confidence: {result['confidence']}")
    print("-" * 80)


Text: Best wireless headphones under $50
Classification: E-commerce
E-commerce Probability: 99.96%
Confidence: 99.96%
--------------------------------------------------------------------------------

Text: Where to buy organic tea online?
Classification: E-commerce
E-commerce Probability: 99.96%
Confidence: 99.96%
--------------------------------------------------------------------------------

Text: iPhone 15 Pro Max deals with contract
Classification: E-commerce
E-commerce Probability: 99.94%
Confidence: 99.94%
--------------------------------------------------------------------------------

Text: Discount codes for Adidas running shoes
Classification: E-commerce
E-commerce Probability: 99.96%
Confidence: 99.96%
--------------------------------------------------------------------------------

Text: Top gaming laptops under $1000
Classification: E-commerce
E-commerce Probability: 98.18%
Confidence: 98.18%
-------------------------------------------------------------------------------